In [1]:
import numpy as np
import matplotlib.pyplot as plt
import datetime

%matplotlib inline

# ALFABURST Survey Notes

### History

* 2017.06.07: end point for initial survey results
* 2017.05.05: increased the clipping threshold to accomodate bright pulses
* 2017.03: reverted to old RFI clipper. New clipper producing spurious results
* 2017.01.24: updated RFI clipper to new method which learns the bandpass, and writes out learned bandpass.
* 2016.12.11: updated RFI clipper to new statistical clipping methods. seems to have a memory leak/running slower than realtime. observations during this time were not successful.

### Logging

* Are there logs/diagnostics proving ALFABURST was actually observing when the logs say it was?
    * cognizeALFA.log is the UPPERLIMIT on the total observing time, this does not account for nodes being down. But, it seems as if abc0, abc1, and abc3 have been consistently up. abc2 has been down most of the time during observation.
    * the LOWERLIMIT on observing time is computed by summing up the time difference between the start time in a dat file and the last recorded event buffer. See alfaburst-surveyScripts/surveyTimeLower.py
    * The cognize ALFA log does not account for when a compute node fails. This is logged on each node as frb_abcX.log

* Are there intermediate diagnostic outputs? e.g. integrated spectra before the single pulse search?
    * no, this should be included in the new pipeline

### Observing Time Upper Limit

* How long have we observed with ALFABURST?
    * based on parseCognizeALFALog.py (from 2015-08-17 to 2017-06-07): 58 days, 8:46:00 (5042760.00 seconds)
    * What about the relibaility of the abcX nodes?
        * abc2 is the most flaky
        * previous to April 2017 the compute node pipeline scripts did not restart processes if they failed, so often observations would be lost. The compute node script now restarts any processes that die, this should lead to near 100% uptime when ALFA is active.
* What does the survey sky coverage look like? How much time has been out of the galactic plan?
    * `./plotSuveryCoverage.py ../data/cognize.pkl --gal_cut=10 --max=200`
        * Total observation time: 5042760 seconds
        * Galaxy Cut Observation time: 2808480 seconds

In [2]:
obsTime = 5042760. #seconds
outOfPlaneObsTime = 2808480. #seconds

print 'Total Obs Time:', str(datetime.timedelta(seconds=obsTime)), '%f hours'%(obsTime/3600.)
print 'Out of Plane Obs Time:', str(datetime.timedelta(seconds=outOfPlaneObsTime))
print 'Precent out of plane:', (outOfPlaneObsTime / obsTime) * 100.

Total Obs Time: 58 days, 8:46:00 1400.766667 hours
Out of Plane Obs Time: 32 days, 12:08:00
Precent out of plane: 55.693310806


### Observing Time Lower Limit

The LOWERLIMIT on observing time is computed by summing up the time difference between the start time in a dat file and the last recorded event buffer.

* July 2015 - Dec 2016: abc3:~/Survey/Scripts/surveyTimeLower.py processed/Beam0_dm_D*
* Jan 2017 - May 2017: abc3:~/Survey/Scripts/surveyTimeLower.py input/Beam0_dm_D*

* Beam 0:
    * July 2015 - Dec 2016: 1261077.891968 s (350.299414 hours)
    * Jan 2017 - May 2017: 212934.346905 s (59.148430 hours)

* Beam 1:
    * July 2015 - Dec 2016: 857322.754677 s (238.145210 hours)
    * Jan 2017 - May 2017: 156960.959905 s (43.600267 hours)

* Beam 2:
    * July 2015 - Dec 2016: 1444512.587052 s (401.253496 hours)
    * Jan 2017 - May 2017: 193889.454858 s (53.858182 hours)

* Beam 3:
    * July 2015 - Dec 2016: 997241.760223 s (277.011600 hours)
    * Jan 2017 - May 2017: 107952.010906 s (29.986670 hours)

* Beam 4:
    * July 2015 - Dec 2016: 471593.221508 s (130.998117 hours)
    * Jan 2017 - May 2017: 262572.440819 s (72.936789 hours)

* Beam 5:
    * July 2015 - Dec 2016: 308866.180089 s (85.796161 hours)
    * Jan 2017 - May 2017: 270820.754567 s (75.227987 hours)

* Beam 6:
    * July 2015 - Dec 2016: 1349927.929134 s (374.979980 hours)
    * Jan 2017 - May 2017: 223508.407003 s (62.085669 hours)

In [3]:
llObsTime = [1261077 + 212934,
             857322 + 156960,
             1444512 + 193889,
             997241 + 107952,
             471593 + 262572,
             308866 + 270820,
             1349927 + 223508]
llObsTime = np.array(llObsTime)

print 'Observing Time Lower Limit:'
for bid,val in enumerate(llObsTime):
    print '  Beam%i: %i s (%f hours)'%(bid, val, val/3600.)
print 'Mean observing time: %f hours'%(llObsTime.mean() / 3600.),
print '(Precent of total Obs time: %.1f%%)'%(100. * llObsTime.mean() / obsTime)

Observing Time Lower Limit:
  Beam0: 1474011 s (409.447500 hours)
  Beam1: 1014282 s (281.745000 hours)
  Beam2: 1638401 s (455.111389 hours)
  Beam3: 1105193 s (306.998056 hours)
  Beam4: 734165 s (203.934722 hours)
  Beam5: 579686 s (161.023889 hours)
  Beam6: 1573435 s (437.065278 hours)
Mean observing time: 322.189405 hours (Precent of total Obs time: 23.0%)


### Survey Coverage

* How much sky have been covered based on the observation time and beam size (for a single pixel, and for the total ALFA beam)? The typical FRB survey rates are quoted in as telescope independent metric of number of square-degree hours.

A single ALFA pixel is approximately 3.8' x 3.3' wide at FWHM ([REF](http://www.naic.edu/~astro/RXstatus/)). Using this we can compute the upper and lower limit survey coverage.

In [4]:
pixelFWHMarea = np.pi * (3.8 / (60. * 2.)) * (3.3 / (60. * 2.)) # pi * r^2, r = (FWHM/2) in degrees
alfaFWHMarea = 7. * pixelFWHMarea

print 'ALFA beam size (using FWHM of 3.8\' x 3.3\'):'
print '  Single beam: %f sq. degrees'%pixelFWHMarea
print '  All beams: %f sq. degrees'%alfaFWHMarea

print 'Sky Coverage using all 7 beams:'
print '  Upper Limit: %f sq. degree hours'%(alfaFWHMarea * (obsTime/3600.))
print '  Lower Limit: %f sq. degree hours'%(np.sum((llObsTime/3600.) * pixelFWHMarea))

ALFA beam size (using FWHM of 3.8' x 3.3'):
  Single beam: 0.002736 sq. degrees
  All beams: 0.019151 sq. degrees
Sky Coverage using all 7 beams:
  Upper Limit: 26.825557 sq. degree hours
  Lower Limit: 6.170129 sq. degree hours


#### Plotting the Survey Coverage

`./plotSuveryCoverage.py ../data/cognize.pkl --gal_cut=10 --max=200 --sampling --NSIDE=256`

Gridding onto a HEALPIX map (NSIDE=256) results in 11401 unique pointings. From (https://lambda.gsfc.nasa.gov/toolbox/tb_pixelcoords.cfm) NSIDE=256 results in 786432 pixels, mean spacing of 0.2290 degrees, which is an area of 1.5978967 x 10^-5 steradian. This is approximately the size of the full ALFA FWHM.

### Sensitivity

* What is the ALFA system temperature? Observing frequency? What is the sensitivity?
    * ALFABURST is 56 MHz centred (usually) at 1375 MHz (Chennamangalam 2017)
    * 'ALFA is a seven-beam system that operates in the 1225–1525 MHz range, with the seven beams arranged in a hexagonal pattern. Each beam is approximately 3.5' wide. The receiver has a cold sky system temperature of ~30 K. The central beam has a gain of ~11 K/Jy , with the peripheral beams having a slightly lower gain. Some of the sidelobes of the ALFA beams are sensitive as well, with the peak of these sidelobes having a loss of only −8.5 dB, i.e., the gain at the sidelobe peak is only 1/7th of that at boresight of the central beam.' (Chennamangalam 2017)
    * Gain, Tsys, SEFD (http://www.naic.edu/alfa/performance/spectral-scans/wide/ , http://www.naic.edu/alfa/performance/)
    
From http://www.naic.edu/alfa/performance/spectral-scans/wide/:

* Central Beam (ID: 0):
    * Gain: 10-12 K/Jy
    * T_sys: 30 K
    * SEFD: 3 Jy
* Side Beam (ID: 1-6):
    * Gain: 9 K/Jy
    * T_sys: 30 K
    * SEFD: 4 Jy

* What is the efficiency of Arecibo?
    * wrapped into the SEFD measurements
    
See [FRB_Event_Rate.ipynb](FRB_Event_Rate.ipynb) for details on the ALFABURST sensitivity to pulses.

### Dispersion Search Space

* ALFABURST search specs:
    * processes 56 MHz (1/8 of the band), 512 channel spectra every 256 microseconds, Stokes I only
    * astroaccelerate dedispersion: 32768 spectra (8.4 seconds) buffers

* We search a maximum DM of 10000, are we saving a large enough buffer if a 10000 DM FRB is detected? yes, just enough for the full band
    * kramer and lorimer eq 5.1 delta t ~ 4.15 * 10^6 ms * (f_ref^-2 - f_chan^-2) * DM, f_ref and f_chan in MHz
    * band: 300 MHz centered at 1375 MHz
    * DM_max = 10000
    * 56 MHz: 1375 - 28 = 1347, 1375 + 28 = 1403 ; delta t = 4.15 * 10^6 m * ( 1347^-2 - 1403^-2) * 10000 = 1789 ms = 1.789 s
    * 280 MHz: 1375 - 140 = 1235, 1375 + 140 = 1515 ; delta t = 4.15 * 10^6 m * ( 1235^-2 - 1515^-2) * 10000 = 9128 ms = 9.128 s

In [5]:
def timeBufferSize(f_start, f_stop, maxdm):
    """Kramer and Lorimer e.q. 5.1
    f_start, f_stop: start (low) and stop (high) frequency, in MHz
    maxdm: maximum search DM
    returns: delay (ms) between start and stop frequency given a maximum DM"""
    bw = f_stop - f_start
    deltat = 4.15 * (10.**6) * (f_start**(-2.) - f_stop**(-2.)) * maxdm
    return deltat

In [6]:
print 'L-band, 56 MHz (1/8th of band):', timeBufferSize(1457., 1513., 10000), 'ms'
print 'L-band, 280 MHz:', timeBufferSize(1233., 1513., 10000), 'ms'

L-band, 56 MHz (1/8th of band): 1420.35077786 ms
L-band, 280 MHz: 9168.59209608 ms


### Predicted Event Rates

See [FRB_Event_Rate.ipynb](FRB_Event_Rate.ipynb) for details on the predicted event rate.